<a href="https://colab.research.google.com/github/ManoharBandam/PSU/blob/master/tapas_finetune_improved_cell_picking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/TAPAS/Fine_tuning_TapasForQuestionAnswering_on_SQA.ipynb#scrollTo=1vfjT1JC_7zI

In [2]:
! rm -r transformers
! git clone https://github.com/huggingface/transformers.git
! cd transformers
! pip install ./transformers

rm: cannot remove 'transformers': No such file or directory
Cloning into 'transformers'...
remote: Enumerating objects: 93421, done.
remote: Counting objects: 100% (308/308), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 93421 (delta 149), reused 223 (delta 103), pack-reused 93113
Receiving objects: 100% (93421/93421), 85.89 MiB | 25.14 MiB/s, done.
Resolving deltas: 100% (68443/68443), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |█████████████████████████

In [3]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=279639 sha256=55de7bd7cb2f0bfc501d479c3f7ff725863d82d0cccf57710821045bf97567c2
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


In [4]:
import pandas as pd
df1 = pd.read_json('sample_data/fetaQA-v1_train.json')
data = pd.json_normalize(df1['data'])
data.head()

,feta_id,table_source_json,page_wikipedia_url,table_page_title,table_section_title,table_array,highlighted_cell_ids,question,answer,source
0,12019,totto_source/train_json/example-4318.json,http://en.wikipedia.org/wiki/Core_fonts_for_th...,Core fonts for the Web,List of fonts and files,"[[File name, Font name, Variants, Last version...","[[1, 1], [1, 3], [9, 1], [9, 3], [20, 1], [20,...",What were the font-versions that were availabl...,The font-versions that were available from Cor...,mturk-approved
1,10691,totto_source/train_json/example-2990.json,http://en.wikipedia.org/wiki/Shruti_Haasan_fil...,Shruti Haasan filmography,Filmography,"[[Title, Year, Role(s), Director, Language(s),...","[[1, 0], [1, 1], [1, 4], [1, 5], [2, 0], [2, 1...",In what films has Shruti Haasan made a cameo a...,Shruti Haasan made a cameo appearance in Tamil...,internal
2,8300,totto_source/train_json/example-599.json,http://en.wikipedia.org/wiki/List_of_Assyrian_...,List of Assyrian kings,Dynasty of Puzur-Ashur (2025–1749 BC),"[[#, Image, King, Reign, Succession, Notes, Re...","[[5, 2], [6, 2], [6, 4], [7, 2], [7, 4]]","On the Assyrian King List, for whom is Sargon ...","On the Assyrian King List, Sargon appears as t...",internal
3,8107,totto_source/train_json/example-406.json,http://en.wikipedia.org/wiki/List_of_Stanley_C...,List of Stanley Cup champions,Active teams,"[[Apps, Team, Wins, Losses, Win %, Years of ap...","[[1, 1], [1, 2], [3, 1], [3, 2]]",How many Stanley Cup championships did Toronto...,The Toronto Maple Leafs has won thirteen Stanl...,internal
4,10564,totto_source/train_json/example-2863.json,http://en.wikipedia.org/wiki/Estadio_El%C3%ADa...,Estadio Elías Aguirre,International matches hosted,"[[Date, Team #1, Res., Team #2, Round, Attenda...","[[1, 1], [1, 2], [1, 3], [1, 4], [5, 1], [5, 2...",What were the first and final Copa América mat...,The game that opened the Elías Aguirre's parti...,internal


In [5]:
data.isnull().sum()


feta_id                 0
table_source_json       0
page_wikipedia_url      0
table_page_title        0
table_section_title     0
table_array             0
highlighted_cell_ids    0
question                0
answer                  0
source                  0
dtype: int64

In [6]:
import ast

def _parse_highlighted_coordinates(answer_coordinate_str):
  """Parses the answer_coordinates of a question.
  Args:
    answer_coordinate_str: A string representation of a Python list of tuple
      strings.
      For example: "['(1, 4)','(1, 3)', ...]"
  """

  try:
    answer_coordinates = []
    # make a list of strings
    # parse each string as a tuple
    for idx in answer_coordinate_str:
      answer_coordinates.append(tuple(idx))
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_coordinate_str)
  answer_coordinates_dtemp = []
  answer_coordinates_dtemp.append(answer_coordinates)
  return answer_coordinates_dtemp

def _parse_answer_text(answer_text):
  """Populates the answer_texts field of `answer` by parsing `answer_text`.
  Args:
    answer_text: A string representation of a Python list of strings.
      For example: "[u'test', u'hello', ...]"
    answer: an Answer object.
  """
  try:
    answer = []
    answer.append(answer_text)

  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_text)

  return answer


data['highlighted_cell_ids'] = data['highlighted_cell_ids'].apply(lambda coords_str: _parse_highlighted_coordinates(coords_str))
data['answer'] = data['answer'].apply(lambda txt: _parse_answer_text(txt))
data['question'] = data['question'].apply(lambda txt: _parse_answer_text(txt))

data.head()

,feta_id,table_source_json,page_wikipedia_url,table_page_title,table_section_title,table_array,highlighted_cell_ids,question,answer,source
0,12019,totto_source/train_json/example-4318.json,http://en.wikipedia.org/wiki/Core_fonts_for_th...,Core fonts for the Web,List of fonts and files,"[[File name, Font name, Variants, Last version...","[[(1, 1), (1, 3), (9, 1), (9, 3), (20, 1), (20...",[What were the font-versions that were availab...,[The font-versions that were available from Co...,mturk-approved
1,10691,totto_source/train_json/example-2990.json,http://en.wikipedia.org/wiki/Shruti_Haasan_fil...,Shruti Haasan filmography,Filmography,"[[Title, Year, Role(s), Director, Language(s),...","[[(1, 0), (1, 1), (1, 4), (1, 5), (2, 0), (2, ...",[In what films has Shruti Haasan made a cameo ...,[Shruti Haasan made a cameo appearance in Tami...,internal
2,8300,totto_source/train_json/example-599.json,http://en.wikipedia.org/wiki/List_of_Assyrian_...,List of Assyrian kings,Dynasty of Puzur-Ashur (2025–1749 BC),"[[#, Image, King, Reign, Succession, Notes, Re...","[[(5, 2), (6, 2), (6, 4), (7, 2), (7, 4)]]","[On the Assyrian King List, for whom is Sargon...","[On the Assyrian King List, Sargon appears as ...",internal
3,8107,totto_source/train_json/example-406.json,http://en.wikipedia.org/wiki/List_of_Stanley_C...,List of Stanley Cup champions,Active teams,"[[Apps, Team, Wins, Losses, Win %, Years of ap...","[[(1, 1), (1, 2), (3, 1), (3, 2)]]",[How many Stanley Cup championships did Toront...,[The Toronto Maple Leafs has won thirteen Stan...,internal
4,10564,totto_source/train_json/example-2863.json,http://en.wikipedia.org/wiki/Estadio_El%C3%ADa...,Estadio Elías Aguirre,International matches hosted,"[[Date, Team #1, Res., Team #2, Round, Attenda...","[[(1, 1), (1, 2), (1, 3), (1, 4), (5, 1), (5, ...",[What were the first and final Copa América ma...,[The game that opened the Elías Aguirre's part...,internal


In [7]:
data.isnull().sum()

feta_id                 0
table_source_json       0
page_wikipedia_url      0
table_page_title        0
table_section_title     0
table_array             0
highlighted_cell_ids    0
question                0
answer                  0
source                  0
dtype: int64

In [8]:
table = pd.DataFrame(data["table_array"][0])
table.columns = table.iloc[0]
table = table.iloc[1: , :].reset_index(drop=True).astype(str)
item = data.iloc[0]
# display(table)
print(item)
print(item.highlighted_cell_ids)

feta_id                                                             12019
table_source_json               totto_source/train_json/example-4318.json
page_wikipedia_url      http://en.wikipedia.org/wiki/Core_fonts_for_th...
table_page_title                                   Core fonts for the Web
table_section_title                               List of fonts and files
table_array             [[File name, Font name, Variants, Last version...
highlighted_cell_ids    [[(1, 1), (1, 3), (9, 1), (9, 3), (20, 1), (20...
question                [What were the font-versions that were availab...
answer                  [The font-versions that were available from Co...
source                                                     mturk-approved
Name: 0, dtype: object
[[(1, 1), (1, 3), (9, 1), (9, 3), (20, 1), (20, 3)]]


In [9]:
table.head()

,File name,Font name,Variants,Last version,Copyright,Sample
0,arial32.exe,"Arial for Windows 9x, NT and Windows 2000","regular, bold, italic, bold italic",version 2.82,Monotype,-
1,Arial.sit.hqx,Arial for Apple Mac OS,"regular, bold, italic, bold italic",version 2.90,Monotype,-
2,arialb32.exe,"Arial Black for Windows 9x, NT and Windows 2000",black,version 2.35,Monotype,-
3,ariblk.exe,Arial Black for Windows 3.1 and 3.11,black,version 2.20,Monotype,-
4,ArialBlack.sit.hqx,Arial Black for Apple Mac OS,black,version 2.35,Monotype,-


In [10]:
import torch
from transformers import TapasTokenizer

# initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/154 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

In [11]:
encoding = tokenizer(table=table, queries=item.question, answer_coordinates=item.highlighted_cell_ids, answer_text=item.answer,
                     truncation=True, padding="max_length", return_tensors="pt")
encoding.keys()

dict_keys(['input_ids', 'labels', 'numeric_values', 'numeric_values_scale', 'token_type_ids', 'attention_mask'])

In [12]:
tokenizer.decode(encoding["input_ids"][0])

'[CLS] what were the font - versions that were available from core fonts for the web project published in 2000? [SEP] file name font name variants last version copyright sample arial32 arial for regular, bold version 2. monotype [EMPTY] arial. arial for regular, bold version 2. monotype [EMPTY] arialb32 arial black black version 2. monotype [EMPTY] ariblk arial black black version 2. monotype [EMPTY] arialblack arial black black version 2. monotype [EMPTY] andale32 andale mono regular version 2. monotype [EMPTY] andalemono andale mono regular version 2. monotype [EMPTY] mtcom. monotype. regular version 1. monotype [EMPTY] courie32 courier new for regular, bold version 2. monotype [EMPTY] couriernew courier new for regular, bold version 2. monotype [EMPTY] comic32. comic sans ms regular, bold version 2. microsoft [EMPTY] comic. exe comic sans ms regular, bold version 1. microsoft [EMPTY] comicsans. comic sans ms regular, bold version 2. microsoft [EMPTY] georgi32 georgia for windows reg

In [13]:
assert encoding["token_type_ids"][0][:,3].sum() == 0

In [14]:
print(item.answer[0])

The font-versions that were available from Core fonts for the Web project were 2.x (e.g. 2.82 for Arial, Times New Roman and Courier New for MS Windows), published in 2000.


In [15]:
# encoding

In [16]:
encoding["token_type_ids"][0][1]

tensor([0, 0, 0, 0, 0, 0, 0])

In [17]:
# encoding["input_ids"]

In [18]:
# data_copy = data.copy()
# data = data.drop(5)
# data=data.reset_index(drop=True)

In [19]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        table = pd.DataFrame(self.df["table_array"][idx]) # TapasTokenizer expects the table data to be text only
          # this means it's the first table-question pair in a sequence
        table.columns = table.iloc[0]
        table = table.iloc[1: , :].reset_index(drop=True)
        # if item.
        print(item)
        encoding = self.tokenizer(table=table, 
                                  queries=item.question, 
                                  answer_coordinates=item.highlighted_cell_ids, 
                                  answer_text=item.answer,
                                  padding="max_length",
                                  truncation=True,
                                  return_tensors="pt"
        )
        # remove the batch dimension which the tokenizer adds 
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        # previous_item = self.df.iloc[idx-1]
        # encoding = self.tokenizer(table=table, 
        #                           queries=[previous_item.question, item.question], 
        #                           answer_coordinates=[previous_item.highlighted_cell_ids, item.highlighted_cell_ids], 
        #                           answer_text=[previous_item.answer, item.answer],
        #                           padding="max_length",
        #                           truncation=True,
        #                           return_tensors="pt"
        # )
        # # use encodings of second table-question pair in the batch
        # encoding = {key: val[-1] for key, val in encoding.items()}
        return encoding

    def __len__(self):
        return len(self.df)

train_dataset = TableDataset(df=data, tokenizer=tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1)

In [20]:
train_dataset[0]["token_type_ids"].shape

feta_id                                                             12019
table_source_json               totto_source/train_json/example-4318.json
page_wikipedia_url      http://en.wikipedia.org/wiki/Core_fonts_for_th...
table_page_title                                   Core fonts for the Web
table_section_title                               List of fonts and files
table_array             [[File name, Font name, Variants, Last version...
highlighted_cell_ids    [[(1, 1), (1, 3), (9, 1), (9, 3), (20, 1), (20...
question                [What were the font-versions that were availab...
answer                  [The font-versions that were available from Co...
source                                                     mturk-approved
Name: 0, dtype: object


torch.Size([512, 7])

In [21]:
train_dataset[1]["input_ids"].shape

feta_id                                                             10691
table_source_json               totto_source/train_json/example-2990.json
page_wikipedia_url      http://en.wikipedia.org/wiki/Shruti_Haasan_fil...
table_page_title                                Shruti Haasan filmography
table_section_title                                           Filmography
table_array             [[Title, Year, Role(s), Director, Language(s),...
highlighted_cell_ids    [[(1, 0), (1, 1), (1, 4), (1, 5), (2, 0), (2, ...
question                [In what films has Shruti Haasan made a cameo ...
answer                  [Shruti Haasan made a cameo appearance in Tami...
source                                                           internal
Name: 1, dtype: object


torch.Size([512])

In [22]:
print(train_dataloader)

In [23]:
batch = next(iter(train_dataloader))

feta_id                                                             12019
table_source_json               totto_source/train_json/example-4318.json
page_wikipedia_url      http://en.wikipedia.org/wiki/Core_fonts_for_th...
table_page_title                                   Core fonts for the Web
table_section_title                               List of fonts and files
table_array             [[File name, Font name, Variants, Last version...
highlighted_cell_ids    [[(1, 1), (1, 3), (9, 1), (9, 3), (20, 1), (20...
question                [What were the font-versions that were availab...
answer                  [The font-versions that were available from Co...
source                                                     mturk-approved
Name: 0, dtype: object
feta_id                                                             10691
table_source_json               totto_source/train_json/example-2990.json
page_wikipedia_url      http://en.wikipedia.org/wiki/Shruti_Haasan_fil...
table_page_titl

In [24]:
batch["input_ids"].shape

torch.Size([3, 512])

In [25]:
batch["token_type_ids"].shape

torch.Size([3, 512, 7])

In [26]:
tokenizer.decode(batch["input_ids"][0])

'[CLS] what were the font - versions that were available from core fonts for the web project published in 2000? [SEP] file name font name variants last version copyright sample arial32 arial for regular, bold version 2. monotype [EMPTY] arial. arial for regular, bold version 2. monotype [EMPTY] arialb32 arial black black version 2. monotype [EMPTY] ariblk arial black black version 2. monotype [EMPTY] arialblack arial black black version 2. monotype [EMPTY] andale32 andale mono regular version 2. monotype [EMPTY] andalemono andale mono regular version 2. monotype [EMPTY] mtcom. monotype. regular version 1. monotype [EMPTY] courie32 courier new for regular, bold version 2. monotype [EMPTY] couriernew courier new for regular, bold version 2. monotype [EMPTY] comic32. comic sans ms regular, bold version 2. microsoft [EMPTY] comic. exe comic sans ms regular, bold version 1. microsoft [EMPTY] comicsans. comic sans ms regular, bold version 2. microsoft [EMPTY] georgi32 georgia for windows reg

In [27]:
#first example should not have any prev_labels set
assert batch["token_type_ids"][0][:,3].sum() == 0

In [28]:
tokenizer.decode(batch["input_ids"][1])

'[CLS] in what films has shruti haasan made a cameo appearance? [SEP] title year role ( s ) director language ( s ) notes ref. hey ram 2000 unknown kamal haasan tamil cameo appearance [EMPTY] hey ram 2000 unknown kamal haasan hindi cameo appearance [EMPTY] luck 2009 ayesha, natasha soham shah hindi [EMPTY] [EMPTY] anaganaga o dheerudu 2011 priya prakash kovelamudi telugu filmfare award for best female debut [EMPTY] dil toh baccha 2011 nikki narang madhur bhandarkar hindi [EMPTY] [EMPTY] 7aum arivu 2011 subha srinivasan ar murugadoss tamil filmfare award for best female debut [EMPTY] oh my friend 2011 siri venu sree raam telugu [EMPTY] [EMPTY] 3 2012 janani aishwarya r. tamil nominated — filmfare award for best [EMPTY] gabbar singh 2012 bhagyalakshmi harish shankar telugu [EMPTY] [EMPTY] balupu 2013 shruti gopichand malineni telugu [EMPTY] [EMPTY] ramaiya vastavaiya 2013 sona prabhu deva hindi [EMPTY] [EMPTY] d - day 2013 suraiya nikhil advani hindi [EMPTY] [EMPTY] ramayya vasthavayya 2

In [29]:
# assert batch["labels"][0].sum() == batch["token_type_ids"][1][:,3].sum()
# print(batch["token_type_ids"][1][:,3].sum())

In [30]:
from transformers import TapasForQuestionAnswering

model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_bias', 'output_weights', 'output_bias', 'column_output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.07, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0): TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   print("Epoch:", epoch)
   for idx, batch in enumerate(train_dataloader):
        # get the inputs;
        print("idx, Batch:",idx,batch)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                       labels=labels)
        loss = outputs.loss
        print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

In [ ]:
# Inference
import collections
import numpy as np

def compute_prediction_sequence(model, data, device):
  """Computes predictions using model's answers to the previous questions."""
  
  # prepare data
  input_ids = data["input_ids"].to(device)
  attention_mask = data["attention_mask"].to(device)
  token_type_ids = data["token_type_ids"].to(device)

  all_logits = []
  prev_answers = None

  num_batch = data["input_ids"].shape[0]
  
  for idx in range(num_batch):
    
    if prev_answers is not None:
        coords_to_answer = prev_answers[idx]
        # Next, set the label ids predicted by the model
        prev_label_ids_example = token_type_ids_example[:,3] # shape (seq_len,)
        model_label_ids = np.zeros_like(prev_label_ids_example.cpu().numpy()) # shape (seq_len,)

        # for each token in the sequence:
        token_type_ids_example = token_type_ids[idx] # shape (seq_len, 7)
        for i in range(model_label_ids.shape[0]):
          segment_id = token_type_ids_example[:,0].tolist()[i]
          col_id = token_type_ids_example[:,1].tolist()[i] - 1
          row_id = token_type_ids_example[:,2].tolist()[i] - 1
          if row_id >= 0 and col_id >= 0 and segment_id == 1:
            model_label_ids[i] = int(coords_to_answer[(col_id, row_id)])

        # set the prev label ids of the example (shape (1, seq_len) )
        token_type_ids_example[:,3] = torch.from_numpy(model_label_ids).type(torch.long).to(device)   

    prev_answers = {}
    # get the example
    input_ids_example = input_ids[idx] # shape (seq_len,)
    attention_mask_example = attention_mask[idx] # shape (seq_len,)
    token_type_ids_example = token_type_ids[idx] # shape (seq_len, 7)
    # forward pass to obtain the logits
    outputs = model(input_ids=input_ids_example.unsqueeze(0), 
                    attention_mask=attention_mask_example.unsqueeze(0), 
                    token_type_ids=token_type_ids_example.unsqueeze(0))
    logits = outputs.logits
    all_logits.append(logits)

    # convert logits to probabilities (which are of shape (1, seq_len))
    dist_per_token = torch.distributions.Bernoulli(logits=logits)
    probabilities = dist_per_token.probs * attention_mask_example.type(torch.float32).to(dist_per_token.probs.device) 

    # Compute average probability per cell, aggregating over tokens.
    # Dictionary maps coordinates to a list of one or more probabilities
    coords_to_probs = collections.defaultdict(list)
    prev_answers = {}
    for i, p in enumerate(probabilities.squeeze().tolist()):
      segment_id = token_type_ids_example[:,0].tolist()[i]
      col = token_type_ids_example[:,1].tolist()[i] - 1
      row = token_type_ids_example[:,2].tolist()[i] - 1
      if col >= 0 and row >= 0 and segment_id == 1:
        coords_to_probs[(col, row)].append(p)

    # Next, map cell coordinates to 1 or 0 (depending on whether the mean prob of all cell tokens is > 0.5)
    coords_to_answer = {}
    for key in coords_to_probs:
      coords_to_answer[key] = np.array(coords_to_probs[key]).mean() > 0.5
    prev_answers[idx+1] = coords_to_answer
    
  logits_batch = torch.cat(tuple(all_logits), 0)
  
  return logits_batch

In [ ]:
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(table.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(table.iat[coordinate])
    answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)